In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import talib                     
import pynance as py             
import quantstats as qs 
import glob
import os


# Make plots a bit nicer
plt.style.use("seaborn-v0_8")
plt.rcParams["figure.figsize"] = (12, 6)


In [5]:

df = pd.read_csv("../data/AAPL.csv")
df.head()


,Date,Close,High,Low,Open,Volume
0,2009-01-02,2.721686,2.730385,2.554037,2.575630,746015200
1,2009-01-05,2.836553,2.884539,2.780469,2.794266,1181608400
2,2009-01-06,2.789767,2.914229,2.770872,2.877641,1289310400
3,2009-01-07,2.729484,2.774170,2.706990,2.753477,753048800
4,2009-01-08,2.780169,2.793666,2.700393,2.712090,673500800


In [6]:

df_news = pd.read_csv("../data/raw_analyst_ratings.csv")
print(df_news['stock'].unique()[:30])
print("Total unique tickers:", df_news['stock'].nunique())

['A' 'AA' 'AAC' 'AADR' 'AAL' 'AAMC' 'AAME' 'AAN' 'AAOI' 'AAON' 'AAP'
 'AAPL' 'AAU' 'AAV' 'AAVL' 'AAWW' 'AAXJ' 'AB' 'ABAC' 'ABAX' 'ABB' 'ABBV'
 'ABC' 'ABCB' 'ABCD' 'ABCO' 'ABCW' 'ABDC' 'ABEV' 'ABG']
Total unique tickers: 6204


In [9]:
files = glob.glob("../data/*.csv")
price_files = [f for f in files if "raw_analyst_ratings" not in f]
price_files

['../data\\AAPL.csv',
 '../data\\AMZN.csv',
 '../data\\GOOG.csv',
 '../data\\META.csv',
 '../data\\MSFT.csv',
 '../data\\NVDA.csv']

In [10]:
dataframes = []

for file in price_files:
    df = pd.read_csv(file)
    ticker = os.path.basename(file).replace(".csv", "")
    df["Ticker"] = ticker
    dataframes.append(df)


In [11]:
all_prices = pd.concat(dataframes, ignore_index=True)
